---
### Import zones

---

In [1]:
import pandas as pd
import numpy as np


---
### Load datasets

---

In [2]:
df_2018 = pd.read_csv('../data/clean_stackoverflow/clean_2018.csv')
df_2019 = pd.read_csv('../data/clean_stackoverflow/clean_2019.csv')
df_2020 = pd.read_csv('../data/clean_stackoverflow/clean_2020.csv')
df_2021 = pd.read_csv('../data/clean_stackoverflow/clean_2021.csv')
df_2022 = pd.read_csv('../data/clean_stackoverflow/clean_2022.csv')

---
### Concat 5 datasets

---

In [4]:
dfs = [df_2018, df_2019, df_2020, df_2021, df_2022]
df = pd.concat(dfs, ignore_index=True)


---
### Get the dev, job and exp dataframes

---

In [10]:
dev_df = df[['Age', 'Gender', 'Sexuality', 'Ethnicity', 'EdLevel','SurveyYear']]
job_df = df[['CompanySize', 'Country', 'Employment', 'DevType(Position)', 'Currency',
        'CurrencySymbol', 'Salary', 'SalaryFreq', 'DollarizedSalary']]
exp_df = df[['YearsCode', 'YearsCodePro', 'OperatingSystem']]

---
### Add some extra columns (ids for primary key, and future foreign keys)

---

In [11]:
n_rows = df.shape[0]
id = np.arange(0, n_rows)

In [12]:
dev_df.insert(0, 'id_dev', dev_df['SurveyYear']*1000000+id, True)
dev_df.insert(7, 'id_job', id, True)
dev_df.insert(8, 'id_exp', id, True)

job_df.insert(0, 'id_job', id, True)
exp_df.insert(0, 'id_exp', id, True)


In [13]:
dev_df.head()

,id_dev,Age,Gender,Sexuality,Ethnicity,EdLevel,SurveyYear,id_job,id_exp
0,2018000000,35 - 44 years old,Male,Straight or heterosexual,White or of European descent,"Bachelor’s degree (BA, BS, B.Eng., etc.)",2018,0,0
1,2018000001,35 - 44 years old,Male,Straight or heterosexual,White or of European descent,"Bachelor’s degree (BA, BS, B.Eng., etc.)",2018,1,1
2,2018000002,18 - 24 years old,Male,Straight or heterosexual,White or of European descent,"Bachelor’s degree (BA, BS, B.Eng., etc.)",2018,2,2
3,2018000003,18 - 24 years old,Male,Straight or heterosexual,White or of European descent,Some college/university study without earning ...,2018,3,3
4,2018000004,35 - 44 years old,Male,Straight or heterosexual,White or of European descent,Some college/university study without earning ...,2018,4,4


In [14]:
exp_df.tail()

,id_exp,YearsCode,YearsCodePro,OperatingSystem
177617,177617,4,NaN,NaN
177618,177618,6,5,Linux-based;Windows
177619,177619,42,33,Windows
177620,177620,50,31,Windows
177621,177621,16,5,Linux-based;Windows


---
### Get unique datasets

---

In [15]:
def col_with_semicolon_to_list_of_unique(df, col):
    """
        Transforms a column of data separated by semicolons into a list with the 
        unique values of the entire column.

        Input:
            df: Dataframe, Dataframe containing desired column
            col: String, name of the column

        Output:
            unique_values_list: list, list of unique values across the entire column
    """
    df_raw = df[col].copy()
    df_raw.dropna(inplace=True)
    df_raw = df_raw.unique().tolist()
    list_with_duplicates = []

    for x in df_raw:
        splitted = x.split(';')
        list_with_duplicates.append(splitted)

    # Plain data with comprehension list
    plain_list = [data for sublist in list_with_duplicates for data in sublist]

    # Unique values
    unique_values_set = set(plain_list)

    # Unique values list
    unique_values_list = list(unique_values_set)

    return unique_values_list

In [16]:
# get list of unique ocurrences

unique_lang = col_with_semicolon_to_list_of_unique(df, 'LanguageHaveWorkedWith')
unique_db = col_with_semicolon_to_list_of_unique(df, 'DatabaseHaveWorkedWith')
unique_framework = col_with_semicolon_to_list_of_unique(df, 'FrameworkHaveWorkedWith')
unique_platform = col_with_semicolon_to_list_of_unique(df, 'PlatformHaveWorkedWith')

# Get number of ocurrences
lang_len = len(unique_lang)
db_len = len(unique_db)
framework_len = len(unique_framework)
platform_len = len(unique_platform)

# Create ids for each new dataframe
lang_idx = np.arange(0, lang_len)
db_idx = np.arange(0, db_len)
framework_idx = np.arange(0, framework_len)
platform_idx = np.arange(0, platform_len)

# Create a new Dataframe for each 
lang_df = pd.DataFrame({'id_lang':lang_idx, 'language':unique_lang})
db_df = pd.DataFrame({'id_db':db_idx, 'database':unique_db})
framework_df = pd.DataFrame({'id_framework':framework_idx, 'framework':unique_framework})
platform_df = pd.DataFrame({'id_platform':platform_idx, 'platform':unique_platform})

In [17]:
lang_df

,id_lang,language
0,0,Go
1,1,Visual Basic 6
2,2,Solidity
3,3,F#
4,4,Scala
5,5,HTML/CSS
6,6,Matlab
7,7,Crystal
8,8,Hack
9,9,Cobol


---
### Dict for each table

---

In [18]:
inv_lang_dict = lang_df['language'].to_dict()
inv_db_dict = db_df['database'].to_dict()
inv_framework_dict = framework_df['framework'].to_dict()
inv_platform_dict = platform_df['platform'].to_dict()

In [19]:
lang_dict = {v:k for k, v in inv_lang_dict.items()}
db_dict = {v:k for k, v in inv_db_dict.items()}
framework_dict = {v:k for k, v in inv_framework_dict.items()}
platform_dict = {v:k for k, v in inv_platform_dict.items()}

In [20]:
lang_dict

{'Go': 0,
 'Visual Basic 6': 1,
 'Solidity': 2,
 'F#': 3,
 'Scala': 4,
 'HTML/CSS': 5,
 'Matlab': 6,
 'Crystal': 7,
 'Hack': 8,
 'Cobol': 9,
 'Delphi/Object Pascal': 10,
 'OCaml': 11,
 'CoffeeScript': 12,
 'Rust': 13,
 'Bash/Shell/PowerShell': 14,
 'C': 15,
 'Elixir': 16,
 'Assembly': 17,
 'CSS': 18,
 'C#': 19,
 'HTML': 20,
 'Erlang': 21,
 'R': 22,
 'LISP': 23,
 'SQL': 24,
 'APL': 25,
 'VB.NET': 26,
 'Python': 27,
 'Java': 28,
 'Delphi': 29,
 'WebAssembly': 30,
 'Perl': 31,
 'Fortran': 32,
 'Ocaml': 33,
 'Kotlin': 34,
 'Other(s):': 35,
 'Dart': 36,
 'Julia': 37,
 'Node.js': 38,
 'COBOL': 39,
 'Groovy': 40,
 'Objective-C': 41,
 'Haskell': 42,
 'Ruby': 43,
 'TypeScript': 44,
 'Lua': 45,
 'JavaScript': 46,
 'C++': 47,
 'Clojure': 48,
 'PowerShell': 49,
 'MATLAB': 50,
 'SAS': 51,
 'VBA': 52,
 'PHP': 53,
 'Swift': 54,
 'Bash/Shell': 55}

---
### Intermediate tables

---

In [21]:
# Language
df_exp_lang = df.assign(LanguageHaveWorkedWith=df['LanguageHaveWorkedWith'].str.split(';')).explode('LanguageHaveWorkedWith')
df_exp_lang = df_exp_lang.reset_index().rename(columns={'index':'id'})
df_exp_lang = df_exp_lang[['id','LanguageHaveWorkedWith']].drop_duplicates().reset_index(drop=True).sort_values('id')
df_exp_lang = df_exp_lang.rename(columns={'LanguageHaveWorkedWith':'language'})
df_exp_lang['id_lang'] = df_exp_lang['language'].map(lang_dict)


In [22]:
# Database
df_exp_db = df.assign(DatabaseHaveWorkedWith=df['DatabaseHaveWorkedWith'].str.split(';')).explode('DatabaseHaveWorkedWith')
df_exp_db = df_exp_db.reset_index().rename(columns={'index':'id'})
df_exp_db = df_exp_db[['id','DatabaseHaveWorkedWith']].drop_duplicates().reset_index(drop=True).sort_values('id')
df_exp_db = df_exp_db.rename(columns={'DatabaseHaveWorkedWith':'database'})
df_exp_db['id_database'] = df_exp_db['database'].map(db_dict)

In [23]:
# Framework
df_exp_framework = df.assign(FrameworkHaveWorkedWith=df['FrameworkHaveWorkedWith'].str.split(';')).explode('FrameworkHaveWorkedWith')
df_exp_framework = df_exp_framework.reset_index().rename(columns={'index':'id'})
df_exp_framework = df_exp_framework[['id','FrameworkHaveWorkedWith']].drop_duplicates().reset_index(drop=True).sort_values('id')
df_exp_framework = df_exp_framework.rename(columns={'FrameworkHaveWorkedWith':'framework'})
df_exp_framework['id_framework'] = df_exp_framework['framework'].map(framework_dict)

In [24]:
# Platform
df_exp_platform = df.assign(PlatformHaveWorkedWith=df['PlatformHaveWorkedWith'].str.split(';')).explode('PlatformHaveWorkedWith')
df_exp_platform = df_exp_platform.reset_index().rename(columns={'index':'id'})
df_exp_platform = df_exp_platform[['id','PlatformHaveWorkedWith']].drop_duplicates().reset_index(drop=True).sort_values('id')
df_exp_platform = df_exp_platform.rename(columns={'PlatformHaveWorkedWith':'platform'})
df_exp_platform['id_platform'] = df_exp_platform['platform'].map(platform_dict)

In [25]:
df_exp_platform.head()

,id,platform,id_platform
0,0,Linux,29.0
1,1,Azure,21.0
2,2,Linux,29.0
3,3,Linux,29.0
4,4,Amazon Echo,4.0


---
### Drop duplicated data from intermediate tables, and rename ids

---

#### Experience-Language

---

In [26]:
df_exp_lang.drop(columns=['language'], inplace=True)
df_exp_lang.rename(columns={'id':'id_exp'}, inplace=True)
df_exp_lang.reset_index(inplace=True, drop=False)
df_exp_lang.rename(columns={'index':'id_exp_lang'}, inplace=True)
df_exp_lang.head()

,id_exp_lang,id_exp,id_lang
0,0,0,46.0
1,1,0,27.0
2,2,0,55.0
3,3,1,19.0
4,4,1,46.0


---
#### experience-db

---

In [27]:
df_exp_db.drop(columns=['database'], inplace=True)
df_exp_db.reset_index(inplace=True, drop=False)
df_exp_db.rename(columns={'id':'id_exp', 'index':'id_exp_db'}, inplace=True)
df_exp_db.head()


,id_exp_db,id_exp,id_database
0,0,0,20.0
1,1,0,1.0
2,2,0,28.0
3,3,1,8.0
4,4,1,26.0


---
#### experience-framework

---

In [28]:
df_exp_framework.drop(columns=['framework'], inplace=True)
df_exp_framework.reset_index(inplace=True, drop=False)
df_exp_framework.rename(columns={'id':'id_exp', 'index':'id_exp_framework'}, inplace=True)
df_exp_framework.head()

,id_exp_framework,id_exp,id_framework
0,0,0,33.0
1,1,1,NaN
2,2,2,30.0
3,3,2,24.0
4,4,3,24.0


---
#### experience-platform

---

In [29]:
df_exp_platform.drop(columns=['platform'], inplace=True)
df_exp_platform.reset_index(inplace=True, drop=False)
df_exp_platform.rename(columns={'id':'id_exp', 'index':'id_exp_platform'}, inplace=True)
df_exp_platform.head()

,id_exp_platform,id_exp,id_platform
0,0,0,29.0
1,1,1,21.0
2,2,2,29.0
3,3,3,29.0
4,4,4,4.0


---
## Save all the tables

---

In [99]:
# fact table (master table)
dev_df.to_csv('../data/final_tables/developer.csv', index=False)

# dimension tables
job_df.to_csv('../data/final_tables/job.csv', index=False)
exp_df.to_csv('../data/final_tables/experience.csv', index=False)

# lv2 dimension tables
lang_df.to_csv('../data/final_tables/language.csv', index=False)
db_df.to_csv('../data/final_tables/database.csv', index=False)
framework_df.to_csv('../data/final_tables/framework.csv', index=False)
platform_df.to_csv('../data/final_tables/platform.csv', index=False)

# intermediate tables
df_exp_lang.to_csv('../data/final_tables/exp_lang.csv', index=False)
df_exp_db.to_csv('../data/final_tables/exp_db.csv', index=False)
df_exp_framework.to_csv('../data/final_tables/exp_framework.csv', index=False)
df_exp_platform.to_csv('../data/final_tables/exp_platform.csv', index=False)